In [236]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import matplotlib as mpl

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

#  <b>Training Models - Synopsis<b>
### 1. Linear Regression ✅
>      • Closed-form 
     • Gradient Descent
### 2. Polynomian Regression ✅
### 3. Learning Curves ✅
### 4. Regularized Linear Models✅

### 5. Logistic Regression ✅
### 6. Softmax Regression 113 150 130

# 1. 📈 Linear Regression

## i) Normal Form
To find the value of θ that minimizes the cost function, there is a closed-form solution
—in other words, a mathematical equation that gives the result directly. This is called
the Normal Equation

#### **θ<sup>^</sup> = (X<sup>T</sup>X )<sup>-1</sup> X<sup>T</sup> y**

In [237]:
X = 2*np.random.rand(100, 1)
y = 4 + 3 * X + np.random.randn(100, 1) # y = mx + c
plt.scatter(X, y, s=8)
plt.show()

In [238]:
X_b = np.c_[np.ones((100, 1)), X]
theta_best = np.linalg.inv(X_b.T.dot(X_b)).dot(X_b.T).dot(y)

In [239]:
y_pred = X_b.dot(theta_best)
plt.scatter(X, y, s=7)
plt.plot(X, y_pred, c="orange")
plt.show()

In [240]:
theta_best

#### 👉 Other Methods to Calculate "theta_best"

In [241]:
# 1. Using Sklearn LinearRegression()

# The LinearRegression class is based on the 
# scipy.linalg.lstsq() function (the name stands for 
# “least squares”)

from sklearn.linear_model import LinearRegression
lin_reg = LinearRegression()
lin_reg.fit(X, y)
print( lin_reg.intercept_, lin_reg.coef_ )

In [242]:
# 2. Using Numpy Least Square Method

theta_best_svd, residuals, rank, s= np.linalg.lstsq(X_b, y, rcond=1e-6)
theta_best

In [243]:
# 3. Using Numpy PseudoInverse of X and dot(y)

# This approach is more efficient than computing the
# Normal Equation, plus it handles edge cases nicely: 
# indeed, the Normal Equation may not work if the matrix 
# XTX(X transpose * X) is not invertible (i.e., singular),  
# such as if m < n or if some features are redundant, but the 
# pseudoinverse is always defined.


np.linalg.pinv(X_b).dot(y)

# The pseudoinverse itself is computed using a standard 
# matrix factorization technique called Singular 
# Value Decomposition (SVD) that can decompose the 
# training set matrix X into the matrix multiplication 
# of three matrices U Σ V^T

#### Computation Complexity
The Normal Equation computes the inverse of X<sup>T </sup> X, which is an (n + 1) × (n + 1)
matrix (where n is the number of features). The computational complexity of inverting
such a matrix is typically about O(n<sup>2.4</sup>) to O(n<sup>3</sup>) (depending on the implementation).
In other words, if you double the number of features, you multiply the computation
time by roughly 2<sup>2.4</sup> = 5.3 to 2<sup>3</sup> = 8.

👉 *Both the Normal Equation and the SVD approach get very slow
when the number of features grows large (e.g., 100,000). On the
positive side, both are linear with regards to the number of instances
in the training set (they are O(m)), so they handle large training
sets efficiently, provided they can fit in memory.*

## ii) Gradient Descent
The general idea of Gradient Descent is to
tweak parameters iteratively in order to minimize a cost function.<br>
The cost function has the shape of a bowl, but it can be an elongated bowl if
the features have very different scales.

#### a) Batch Gradient Descent
It is simply the derivative term, which is like asking "what is the slope of the mountain uder my feet if I face east?..and the west and so on..for all directions.

Notice that this formula involves calculations over the full training
set X, at each Gradient Descent step! This is why the algorithm is
called *Batch Gradient Descent*: it uses the whole batch of training
data at every step . As a result it is terribly slow on very large training
sets . Training a Linear Regression model when there are hundreds
of thousands of features is much faster using Gradient
Descent than using the Normal Equation or SVD decomposition.

##### 👉 Gradient Descent step

#### **θ** <sup>(next step)</sup> = **θ** − η ∇<sub>θ</sub>MSE(**θ**)
Once you have the gradient vector, which points uphill, just go in the opposite direction
to go downhill. This means subtracting ∇<sub>θ</sub>MSE(**θ**) from **θ**. This is where the
learning rate η comes into play: multiply the gradient vector by η to determine the
size of the downhill step

In [244]:
eta = 0.1 #learning rate
n_iterations = 1000
m = X_b.shape[0]
bt_theta=np.array([0,0]) #array storing regression values
theta = [[0],[0]]
for iteration in range(n_iterations):
    mse = (2/m)*(X_b.T.dot(X_b.dot(theta) - y))
    theta = theta - eta * mse
    bt_theta = np.c_[bt_theta, theta] #data plotted at last
theta

Note: *To find a good learning rate, you can use grid search (see Chapter 2). However, you
may want to limit the number of iterations so that grid search can eliminate models
that take too long to converge.*

#### b) Stochastic Gradient Descent (SGD)
Batch Gradient Descent uses the whole
training set to compute the gradients at every step, which makes it very slow when
the training set is large. At the opposite extreme, Stochastic Gradient Descent just
picks a random instance in the training set at every step and computes the gradients
based only on that single instance. Obviously this makes the algorithm much faster
since it has very little data to manipulate at every iteration. It also makes it possible to
train on huge training sets, since only one instance needs to be in memory at each
iteration.<br>
On the other hand, due to its stochastic (i.e., random) nature, this algorithm is much
less regular than Batch Gradient Descent:

In [245]:
n_epochs = 50
t0, t1 = 5, 10
def learning_schedule(t):
    return t0 / (t + t1)

theta = np.random.randn(2,1)
sc_theta = np.array([0,0])
for epoch in range(n_epochs):
    for i in range(m):
        random_index = np.random.randint(m)
        xi = X_b[random_index:random_index+1]
        yi = y[random_index: random_index+1]
        gradients = 2 * xi.T.dot(xi.dot(theta) - yi)
        eta = learning_schedule(epoch * m + i)
        theta = theta - eta *  gradients
    sc_theta = np.c_[sc_theta,theta]

theta

##### Perform SGD using Sklearn

In [246]:
from sklearn.linear_model import SGDRegressor
sgd_reg = SGDRegressor(max_iter=1000, tol=1e-3, penalty=None, eta0=0.1)
sgd_reg.fit(X, y.ravel())

In [247]:
sgd_reg.intercept_, sgd_reg.coef_

#### c) Mini-batch Gradient Descent
at each step, instead of computing the gradients based on the full training
set (as in Batch GD) or based on just one instance (as in Stochastic GD), Minibatch
GD computes the gradients on small random sets of instances called minibatches.
The main advantage of Mini-batch GD over Stochastic GD is that you can
get a performance boost from hardware optimization of matrix operations, especially
when using GPUs

In [248]:
plt.figure(figsize=(7,5))
plt.plot(sc_theta[0,1:], sc_theta[1,1:],"rs-", label="Stochastic",linewidth=1)
plt.plot(bt_theta[0,10:], bt_theta[1,10:],"bo-", label="Batch",linewidth=1)
plt.legend()
plt.show()

<table><tr><th>Algorithm</th> <th>Large m</th> <th>Out-of-core support</th> <th>Large n</th> <th> Hyperparams</th> <th>Scaling required</th> <th> Scikit-Learn</th></tr>
<tr><td>Normal Equation</td> <td> Fast </td> <td> No</td><td> Slow </td><td>0</td><td> No</td><td> n/a</td></tr>
<tr><td>SVD</td><td> Fast</td><td> No</td><td> Slow</td><td> 0 </td><td>No</td><td> LinearRegression</td></tr>
<td>Batch GD</td><td> Slow </td><td>No</td><td> Fast</td><td> 2</td><td> Yes</td><td> SGDRegressor</td>
<tr><td>Stochastic GD</td><td> Fast</td><td> Yes</td><td> Fast</td><td> ≥2</td><td> Yes</td><td> SGDRegressor</td></tr>
<tr><td>Mini-batch GD</td><td> Fast</td><td> Yes </td><td>Fast</td><td> ≥2</td><td> Yes</td><td> SGDRegressor</td></tr></table>

# 2. ⤴️ Polynomial Regression
Surprisingly, a simple way to use a linear model to fit nonlinear data is to
add powers of each feature as new features, then train a linear model on this extended
set of features. This technique is called *Polynomial Regression.*

In [249]:
# lets take a test dataset
m = 200
X = 6 * np.random.rand(m, 1) - 3 
y = 0.5 * X**2 + X + 2 +  np.random.randn(m, 1) 

# here, random.randn is used to include some noice in the data
# try to remove this term to get more understanding


In [250]:
plt.scatter(X, y, alpha=0.8)

#### Adding 2nd Degree Polynommial as new feature
As a straight line will never fit this data properly. So, use scikit-learn's to transform our training data, adding the square(2nd degree polynomial) of each feature in the traiing set as new features.

In [251]:
from sklearn.preprocessing import PolynomialFeatures
poly_features = PolynomialFeatures(degree=2, include_bias= False)
X_poly = poly_features.fit_transform(X)
X[0], X_poly[0]

In [252]:
# Now training a linear model on polynomial function 

lin_reg = LinearRegression()
lin_reg.fit(X_poly, y)
lin_reg.intercept_, lin_reg.coef_

In [253]:
pred_test = np.arange(-3, 3.6, 0.2)
pred_line = []
for i in range(len(pred_test)):
    pred_line.append(lin_reg.coef_[0, 0] * pred_test[i] +  lin_reg.coef_[0, 1] * pred_test[i]**2 +  lin_reg.intercept_ )
plt.plot(pred_test, pred_line, "r-", linewidth = 4, alpha=0.9)
plt.scatter(X, y, alpha=0.7)
plt.show()

*Note that when there are multiple features, Polynomial Regression is capable of finding
relationships between features (which is something a plain Linear Regression
model cannot do). This is made possible by the fact that PolynomialFeatures also
adds all combinations of features up to the given degree. For example, if there were
two features a and b, PolynomialFeatures with degree=3 would not only add the
features a2, a3, b2, and b3, but also the combinations ab, a2b, and ab2.*

# 3) 🌊Learning Curve
In Chapter 2 you used cross-validation to get an estimate of a model’s generalization
performance.
Another way is to look at the learning curves: these are plots of the model’s performance
on the training set and the validation set as a function of the training set size
(or the training iteration). To generate the plots, simply train the model several times
on different sized subsets of the training set.

In [254]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

def plot_learning_curve(model, X, y):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

    train_error = []
    test_error = []
    for a in range(1, len(X_train)):
        model.fit(X_train[:a], y_train[:a])
        X_train_pred = model.predict(X_train[:a])
        X_test_pred = model.predict(X_test)
        train_error.append(mean_squared_error(y_train[:a], X_train_pred))
        test_error.append(mean_squared_error(y_test, X_test_pred))
    return train_error, test_error

In [255]:
train_error, test_error = plot_learning_curve(lin_reg, X_poly, y)
plt.plot(np.sqrt(train_error[8:]), "r-+", linewidth=2, label="train")
plt.plot(np.sqrt(test_error[8:]), "b-", linewidth = 3, label="test")
plt.legend()
plt.xlabel("Epoch")
plt.ylabel("MeanSquaredError")
plt.show()

### ⚠️ CAUTION ! 
> If your model is **underfitting** the training data, adding more training
examples will not help. You need to use a more complex model
or come up with better features.<br>
> One way to improve an **overfitting** model is to feed it more training
data until the validation error reaches the training error.

### The Bias/Variance Tradeoff
An important theoretical result of statistics and Machine Learning is the fact that a
model’s generalization error can be expressed as the sum of three very different
errors:
#### Bias
This part of the generalization error is due to wrong assumptions, such as assuming
that the data is linear when it is actually quadratic. A high-bias model is most
likely to underfit the training data.10
#### Variance
This part is due to the model’s excessive sensitivity to small variations in the
training data. A model with many degrees of freedom (such as a high-degree polynomial
model) is likely to have high variance, and thus to overfit the training
data.
#### Irreducible error
This part is due to the noisiness of the data itself. The only way to reduce this
part of the error is to clean up the data (e.g., fix the data sources, such as broken
sensors, or detect and remove outliers).
Increasing a model’s complexity will typically increase its variance and reduce its bias.
Conversely, reducing a model’s complexity increases its bias and reduces its variance.
This is why it is called a tradeoff.

# 4) 📇 Regularized Linear Models
a good way to reduce overfitting is to regularize the
model (i.e., to constrain it): the fewer degrees of freedom it has, the harder it will be
for it to overfit the data. For example, a simple way to regularize a polynomial model
is to reduce the number of polynomial degrees.

## a) Ridge Regression
Ridge Regression (also called Tikhonov regularization) is a regularized version of Linear
Regression: a regularization term equal to <br> **αΣ<sup>n</sup><sub>i = 1</sub>
 θ<sub>i</sub><sup>2</sup>** <br> is added to the cost function.
This forces the learning algorithm to not only fit the data but also keep the model
weights as small as possible. Note that the regularization term should only be added
to the cost function during training.<br>
<h4> Ridge Regression Cost Form: </h4>

#### **J(θ) = MSE(θ) + αΣ<sup>n</sup><sub>i=1 </sub> θ<sub>i</sub><sup>2</sup>**

In [256]:
#Perform Ridge Regression using Ridge Model

from sklearn.linear_model import Ridge
ridge_reg = Ridge(alpha=1000, solver="cholesky")
ridge_reg.fit(X, y)
ridge_pred = ridge_reg.predict(pred_test.reshape(-1, 1))
plt.plot(pred_test, ridge_pred, c="r", label="alpha=1000")

ridge_reg = Ridge(alpha=100, solver="cholesky")
ridge_reg.fit(X, y)
ridge_pred = ridge_reg.predict(pred_test.reshape(-1, 1))
plt.plot(pred_test, ridge_pred, c="g", label="alpha=100")

ridge_reg = Ridge(alpha=0, solver="cholesky")
ridge_reg.fit(X, y)
ridge_pred = ridge_reg.predict(pred_test.reshape(-1, 1))
plt.plot(pred_test, ridge_pred, c="b", label="alpha=0")

plt.legend()
plt.plot(pred_test, ridge_pred)
plt.scatter(X, y, alpha=0.2)

In [257]:
# Ridge Regression using SGDRegressor panelty="l2"

sgd_reg = SGDRegressor( penalty="l2")
sgd_reg.fit(X, y.ravel())
sgd_reg.predict([[-3]])

In [258]:
# making it degree 10 features

poly_features = PolynomialFeatures(degree=10, include_bias= False)
X_poly = poly_features.fit_transform(X)
X[0], X_poly[0]

# scaling the features

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_10_scaled = scaler.fit_transform(X_poly.astype(np.float64))

# Ridge Regression Applied

ridge_reg = Ridge(alpha=1e-100, solver="cholesky")
ridge_reg.fit(X_10_scaled, y)
ridge_pred_10 = ridge_reg.predict(X_10_scaled)
plt.scatter(X, ridge_pred_10, s=5, c="r", label="alpha=1e-100")

ridge_reg = Ridge(alpha=100, solver="cholesky")
ridge_reg.fit(X_10_scaled, y)
ridge_pred_10 = ridge_reg.predict(X_10_scaled)
plt.scatter(X, ridge_pred_10, s=3, c="b", label="alpha=100")

ridge_reg = Ridge(alpha=1, solver="cholesky")
ridge_reg.fit(X_10_scaled, y)
ridge_pred_10 = ridge_reg.predict(X_10_scaled)
plt.scatter(X, ridge_pred_10, s=3, c="g", label="alpha=1")

plt.legend()
plt.scatter(X, y, alpha=0.2)

#increasing the alpha will increase the overfitting

## b) Lasso Regression
Least Absolute Shrinkage and Selection Operator Regression (simply called Lasso
Regression) is another regularized version of Linear Regression: just like Ridge
Regression, it adds a regularization term to the cost function, but it uses the ℓ1 norm
of the weight vector instead of half the square of the ℓ2 norm

<h4> Lasso Regression Cost Form: </h4>

#### **J(θ) = MSE(θ) + αΣ<sup>n</sup><sub>i = 1</sub>|θ<sub>i</sub>|**

In [259]:
from sklearn.linear_model import Lasso
lasso_reg = Lasso(alpha=.2)
lasso_reg.fit(X_10_scaled, y)

lasso_pred = lasso_reg.predict(X_10_scaled)
plt.scatter(X, lasso_pred, c="r", marker="+", label="alpha=0.2")
plt.scatter(X, y, alpha=0.2)
plt.legend()


In [260]:
# can also be applied using SGDRegressor(penalty="l1")

## c) Elastic Net
Elastic Net is a middle ground between Ridge Regression and Lasso Regression. The
regularization term is a simple mix of both Ridge and Lasso’s regularization terms,
and you can control the mix ratio r. When r = 0, Elastic Net is equivalent to Ridge
Regression, and when r = 1, it is equivalent to Lasso Regression
#### **J(θ) = MSE(θ) + αΣ<sup>n</sup><sub>i = 1</sub>|θ<sub>i</sub>| + (1-r)/2 *  αΣ<sup>n</sup><sub>i=1 </sub> θ<sub>i</sub><sup>2</sup>**

In [261]:
from sklearn.linear_model import ElasticNet
elastic_net = ElasticNet(alpha=0.8, l1_ratio=0.5)
elastic_net.fit(X_10_scaled, y)
elastic_pred = elastic_net.predict(X_10_scaled)

plt.scatter(X, elastic_pred, c="r", marker="+", label=f"alpha={elastic_net.alpha}")
plt.scatter(X, y, alpha=0.2)
plt.legend()
plt.plot()

# 5) 🚫Early Stopping
A very different way to regularize iterative learning algorithms such as Gradient
Descent is to stop training as soon as the validation error reaches a minimum. This is
called early stopping.

In [281]:
from sklearn.base import clone
from sklearn.pipeline import Pipeline

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.5)

#prepare the data

poly_scaled = Pipeline([
    ("poly_features", PolynomialFeatures(degree=90, include_bias=False)),
    ("std_scaler", StandardScaler())
])
X_train_poly_scaled = poly_scaled.fit_transform(X_train)
X_val_poly_scaled = poly_scaled.transform(X_test)

sgd_reg = SGDRegressor(max_iter=1, tol=-np.infty, warm_start=True, penalty=None, learning_rate="constant", eta0=0.0005)

minimum_val_error = float("inf")
best_epoch = None
best_model = None
for epoch in range(1000):
    sgd_reg.fit(X_train_poly_scaled, y_train.ravel())
    y_val_predict = sgd_reg.predict(X_val_poly_scaled)
    val_error = mean_squared_error(y_test, y_val_predict)
   
    if val_error < minimum_val_error:
        minimum_val_error = val_error
        best_epoch = epoch
        best_model = clone(sgd_reg)
        best_prediction = y_val_predict
        

In [282]:
best_model.fit(X_val_poly_scaled, y_test.ravel())
prediction = best_model.predict(X_val_poly_scaled)
plt.scatter(X_test, y_test)
plt.scatter(X_test, best_prediction)
plt.show()

In [283]:
best_epoch, minimum_val_error, mean_squared_error(y_test, best_prediction)

# 6) 🪵Logistic Regression
Logistic Regression (also called Logit Regression) is commonly
used to estimate the probability that an instance belongs to a particular class
(e.g., what is the probability that this email is spam?). If the estimated probability is
greater than 50%, then the model predicts that the instance belongs to that class
(called the positive class, labeled “1”), or else it predicts that it does not (i.e., it
belongs to the negative class, labeled “0”). This makes it a binary classifier.

### Logistic Function

In [291]:
y = np.arange(-10, 10, 0.4)
X = 1/(1+np.exp(-y))
plt.plot(y, X,  "b", label=("sigmoid fun."))
plt.grid("on")
plt.legend()

### Decision Boundary
Let’s use the iris dataset to illustrate Logistic Regression. This is a famous dataset that
contains the sepal and petal length and width of 150 iris flowers of three different
species: Iris-Setosa, Iris-Versicolor, and Iris-Virginica

In [293]:
from sklearn import datasets
iris = datasets.load_iris()
list(iris.keys())

In [295]:
X = iris["data"][:, 3:] # petal_width
y = (iris["target"] == 2).astype(int) # If Iris_Virginica, else 0

In [296]:
from sklearn.linear_model import LogisticRegression

log_reg = LogisticRegression()
log_reg.fit(X, y)

In [305]:
X_new = np.linspace(0, 3, 1000).reshape(-1, 1)
y_proba = log_reg.predict_proba(X_new)
plt.plot(X_new, y_proba[:, 1], "g-", label="Iris_Virginica")
plt.plot(X_new, y_proba[:, 0], "b--", label="Not-Iris-Virginica")
plt.scatter(X, y, marker="+", c="red")
plt.legend()
plt.show()

### 🗒️Note:
>The hyperparameter controlling the regularization strength of a
Scikit-Learn LogisticRegression model is not alpha (as in other
linear models), but its inverse: C. The higher the value of C, the less
the model is regularized.

# 6) 🥤 Softmax Regression
The Logistic Regression model can be generalized to support multiple classes directly,
without having to train and combine multiple binary classifiers . This is called Softmax Regression, or Multinomial Logistic Regression.

In [306]:
X = iris["data"][:, (2, 3)] # petal length, petal width
y = iris["target"]

softmax_reg = LogisticRegression(multi_class="multinomial", solver="lbfgs", C=10)
softmax_reg.fit(X, y)

In [307]:
softmax_reg.predict([[5, 2]])

# ... completed 😉😘😮‍💨